In [1]:
import numpy as np
from numpy.random import default_rng
from hyppo.independence import FriedmanRafsky
import csv

_OUT_PATH = './output_folder/output.csv' # output location

In [2]:
# Helper method: format the input for the Friedman Rafsky test.
def combine_then_label(x1, x2):
    return_X = np.concatenate([x1, x2])

    n1 = x1.shape[0]
    n2 = x2.shape[0]

    return_Y = np.repeat([1, 2], [n1, n2])

    return return_X, return_Y

In [3]:
# Helper method: select methods for data simulation

def get_data_simulator_list():
    # parameters for data simulators
    shape_1 = (10, 2)
    shape_2 = (100, 2)
    mean = [0, 0]
    cov = [[1, 0], [0, 100]]

    # methods from numpy (np)
    np_method_names = [
        ("zeros", {"shape": shape_1}),
        ("ones", {"shape": shape_1, "dtype": np.int8})
    ]

    # methods from numpy.random.default_rng
    rng_method_names = [
        ("uniform", {"low": 0, "high": 1000, "size": shape_1}),
        ("uniform", {"low": 2000, "high": 5000, "size": shape_1}),
        ("uniform", {"low": -1000, "high": 1000, "size": shape_1}),
        ("uniform", {"low": -1000, "high": 1000, "size": shape_2}),
        ("multivariate_normal", {"mean": mean, "cov": cov, "size": shape_1[0]}),
        ("multivariate_normal", {"mean": mean, "cov": cov, "size": shape_2[0]})
    ]

    # list to hold data generation methods
    base_datasets_method_components = []

    # get the methods that are sourced from Numpy object
    for method, params in np_method_names:

        source_object_string = "np"
        method_components = {
            "name" : method,
            "params" : params,
            "source_object" : source_object_string,
            "n" : params["shape"][0],
        }
        base_datasets_method_components.append(method_components)

    # get the methods that are sourced from Numpy.Random.default_generator
    for method, params in rng_method_names:
        if(method == "uniform"):
            n = params["size"][0]
        if(method== "multivariate_normal"):
            n = params["size"]

        source_object_string = "rng"
        method_components = {
            "name" : method,
            "params" : params,
            "source_object" : source_object_string,
            "n" : n,
        }
        base_datasets_method_components.append(method_components)

    return base_datasets_method_components

In [4]:
# Helper method: print info about the data being tested
def report_state(c1, c2):
    p = c1
    print(f"Running test with pair:")
    print(
        f"\t- {p['name']}({(p['params'])}), seed = {p['seed']}"
    )
    p = c2
    print(
        f"\t- {p['name']}({(p['params'])}), seed = {p['seed']}"
    )


In [5]:
# Helper method: run Friednman Rafsky and output the following:
# - pvalue
# - corrected test statistics
# - uncorrected test statistic
# - params that generated the data

def make_data(components, seed):
    module_map = {
        "rng": default_rng(seed),
        "np" : np
    }
    components["seed"] = seed

    source_object = module_map[components["source_object"]]
    method_name = components["name"]
    params = components["params"]

    data = getattr(source_object, method_name)(**params)
    return data

def run_FR_simtest(components_1, components_2, independence_expected):
    seed_1 = 402
    seed_2 = 404


    x1 = make_data(components_1, seed_1)
    x2 = make_data(components_2, seed_2)
    
    report_state(components_1, components_2)

    x,y = combine_then_label(x1, x2)
    method = FriedmanRafsky()
    test_rslt = method.test(x,y)
    statistic_rslt = method.statistic(x,y) # uncorrected test statistic

    print(f"test method output: \n\tstatistic: {test_rslt[0]}\n\tpvalue: {test_rslt[1]}")
    print(f"statisc method output: {statistic_rslt}")
    
    # Result to return to be written to csv
    row = {
        "test_rslt_statistic": test_rslt[0], 
        "test_rslt_pvalue": test_rslt[1], 
        "statistic_rslt": statistic_rslt,
        "independence_excepted": independence_expected,
        "n": components_1['n'],
        "m": components_2['n'],
        "dataset_1_info": (components_1),  # (type, parameters, n, seed)"
        "dataset_2_info": (components_2)   # (type, parameters, m, seed)"
    }

    return row

In [6]:
# Helper method: write to csv output
def save_results(rows):

    # each row is stored as a dict, so use the keys as field names
    field_names = rows[0].keys()
    print(f'Creating csv with field names:\n\t {field_names}')

    with open(_OUT_PATH, 'w') as fp:
        writer = csv.DictWriter(
            fp, fieldnames=field_names, 
            delimiter=',', 
            quotechar='"', 
            quoting=csv.QUOTE_MINIMAL
        )
        writer.writeheader()
        writer.writerows(rows)
    

In [16]:
def main_loop():
    self_compare_list = [
        "uniform",
        "multivariate"
    ]

    output_rows = []

    simulator_components_list = get_data_simulator_list()


    for i in range(len(simulator_components_list)):
        simulator_components = simulator_components_list[i]
        method_name = simulator_components["name"]

        if(method_name in self_compare_list):
            print("Running self_compare test:")
            output_rows.append(
                run_FR_simtest(
                    components_1=simulator_components,
                    components_2=simulator_components,
                    independence_expected=False
                )
            )
        for j in range(i, len(simulator_components_list)):
            if(j != i):
                simulator_components_2 = simulator_components_list[j]
                output_rows.append(
                    run_FR_simtest(
                        components_1=simulator_components,
                        components_2=simulator_components_2,
                        independence_expected=True
                    )
                )

    if(not output_rows):
        print("output_rows is empty")
    else:
        save_results(output_rows)


In [17]:
main_loop()

Running test with pair:
	- zeros({'shape': (10, 2)}), seed = 402
	- ones({'shape': (10, 2), 'dtype': <class 'numpy.int8'>}), seed = 404
test method output: 
	statistic: 5.512511590501951
	pvalue: 0.000999000999000999
statisc method output: 20
Running test with pair:
	- zeros({'shape': (10, 2)}), seed = 402
	- uniform({'low': 0, 'high': 1000, 'size': (10, 2)}), seed = 404
test method output: 
	statistic: -0.038712824988120315
	pvalue: 0.6353646353646354
statisc method output: 11
Running test with pair:
	- zeros({'shape': (10, 2)}), seed = 402
	- uniform({'low': 2000, 'high': 5000, 'size': (10, 2)}), seed = 404
test method output: 
	statistic: 0.03781113549437483
	pvalue: 0.6243756243756243
statisc method output: 11
Running test with pair:
	- zeros({'shape': (10, 2)}), seed = 402
	- uniform({'low': -1000, 'high': 1000, 'size': (10, 2)}), seed = 404
test method output: 
	statistic: 0.5519409700063341
	pvalue: 0.3776223776223776
statisc method output: 12
Running test with pair:
	- zeros({'

In [18]:
# Checking the data
import pandas as pd

df = pd.read_csv(_OUT_PATH)

In [19]:
# Were there any tests where statistic was greater than 'm+n'?
df[df['m']+df['n'] < df['statistic_rslt']]

,test_rslt_statistic,test_rslt_pvalue,statistic_rslt,independence_excepted,n,m,dataset_1_info,dataset_2_info


In [20]:
# Were there any tests where statistic was less than 2?
df[2 > df['statistic_rslt']]

,test_rslt_statistic,test_rslt_pvalue,statistic_rslt,independence_excepted,n,m,dataset_1_info,dataset_2_info


In [21]:
# Tests statistcs within expected range
df[2 <= df['statistic_rslt']][df['m']+df['n'] >= df['statistic_rslt']]

,test_rslt_statistic,test_rslt_pvalue,statistic_rslt,independence_excepted,n,m,dataset_1_info,dataset_2_info
0,5.512512,0.000999,20,True,10,10,"{'name': 'zeros', 'params': {'shape': (10, 2)}...","{'name': 'ones', 'params': {'shape': (10, 2), ..."
1,-0.038713,0.635365,11,True,10,10,"{'name': 'zeros', 'params': {'shape': (10, 2)}...","{'name': 'uniform', 'params': {'low': 0, 'high..."
2,0.037811,0.624376,11,True,10,10,"{'name': 'zeros', 'params': {'shape': (10, 2)}...","{'name': 'uniform', 'params': {'low': 2000, 'h..."
3,0.551941,0.377622,12,True,10,10,"{'name': 'zeros', 'params': {'shape': (10, 2)}...","{'name': 'uniform', 'params': {'low': -1000, '..."
4,-2.595487,1.000000,11,True,10,100,"{'name': 'zeros', 'params': {'shape': (10, 2)}...","{'name': 'uniform', 'params': {'low': -1000, '..."
5,0.532043,0.441558,12,True,10,10,"{'name': 'zeros', 'params': {'shape': (10, 2)}...","{'name': 'multivariate_normal', 'params': {'me..."
6,-1.946113,0.996004,13,True,10,100,"{'name': 'zeros', 'params': {'shape': (10, 2)}...","{'name': 'multivariate_normal', 'params': {'me..."
7,-0.032334,0.647353,11,True,10,10,"{'name': 'ones', 'params': {'shape': (10, 2), ...","{'name': 'uniform', 'params': {'low': 0, 'high..."
8,-0.003919,0.628372,11,True,10,10,"{'name': 'ones', 'params': {'shape': (10, 2), ...","{'name': 'uniform', 'params': {'low': 2000, 'h..."
9,0.502449,0.415584,12,True,10,10,"{'name': 'ones', 'params': {'shape': (10, 2), ...","{'name': 'uniform', 'params': {'low': -1000, '..."
